# Machine vs Machine

**CS5483 Data Warehousing and Data Mining**
___

<a title="Freddycastillo9871, CC BY-SA 4.0 &lt;https://creativecommons.org/licenses/by-sa/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Terminator_2.png"><img width="512" alt="Terminator 2" src="https://upload.wikimedia.org/wikipedia/commons/3/38/Terminator_2.png"></a>

In [1]:
%reset -f
from IPython import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# produce vector inline graphics
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
from weka.core import dataset
import weka.core.jvm as jvm
from weka.core.converters import Loader
from weka.classifiers import Classifier, Evaluation, SingleClassifierEnhancer
from sklearn import ensemble, tree
from weka.core.classes import Random
from weka.core.classes import complete_classname
from sklearn import ensemble
from scipy.io import arff
import urllib.request
import io

In this notebook, we will try to build the best machine to classify the image segmentation dataset:
- `segment-challenge.arff` for training, and
- `segment-test.arff` for testing.

## Individual Classification Algorithms

Weka provides a KnowledgeFlow interface to flow data through a learning algorithm. The following is a demo for training and testing a J48 decision tree.

In [2]:
display.IFrame(src="https://cityuhk-lms.ap.panopto.com/Panopto/Pages/Embed.aspx?id=3f33d95a-c5c2-4893-925b-acd10064acec&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all", height=450, width=805)

Similar to the other interfaces, the evaluation results can be saved to files as demonstrated below. You may also load an existing template using a button in the toolbar on the top right-hand corner, as illustrated in the video below.

In [3]:
display.IFrame(src="https://cityuhk-lms.ap.panopto.com/Panopto/Pages/Embed.aspx?id=e60913e2-128b-4e40-962d-acd1006b8347&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all", height=450, width=805)

Unlike the other interfaces, KnowledgeFlow interface can train a classifier incrementally as more and more data becomes available. For more details, refer to the manual [here](https://www.cs.waikato.ac.nz/ml/weka/Witten_et_al_2016_appendix.pdf) and the [video tutorial](https://www.futurelearn.com/info/courses/more-data-mining-with-weka/0/steps/29106) by Witten.

Furthermore, unlike the Explorer interface, one can flow data through multiple classification algorithms simultaneously as illustrated below:

In [4]:
display.IFrame(src="https://cityuhk-lms.ap.panopto.com/Panopto/Pages/Embed.aspx?id=0d3c11b4-7809-45c7-a208-acd1006ea2f1&autoplay=false&offerviewer=true&showtitle=true&showbrand=false&start=0&interactivity=all",  height=450, width=805)

**Exercise** Follow the video above to add other classifiers: `IBk`, `PART`, and `JRIP`. Record their fractional accuracies in the dictionary `performance` as follows:

```Python
performance = {'J48': 0.961728,
               'IBk': ___,
               'ZeroR': ___,
               'OneR': ___,
               'PART': ___,
               'JRIP': ___}
```

Use the default parameters.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
performance

In [ ]:
# tests

## Ensemble Methods

Next, we use the ensemble methods implemented in scikit-learn and Weka to train armies of classifiers.

### Data Preparation

The following load the data for scikit-learn.

In [ ]:
def load_url(url):
    ftpstream = urllib.request.urlopen(url)
    df = pd.DataFrame(arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))[0])
    return df.loc[:,lambda df: ~df.columns.isin(['class'])], df['class'].astype(str)

weka_data_path = 'https://raw.githubusercontent.com/Waikato/weka-3.8/master/wekadocs/data/'
X_train, Y_train = load_url(weka_data_path + 'segment-challenge.arff')
X_test, Y_test = load_url(weka_data_path + 'segment-test.arff')

The following loads the data for `python-weka-wrapper`.

In [ ]:
jvm.start()
loader = Loader(classname="weka.core.converters.ArffLoader")
trainset = loader.load_url(
    weka_data_path +
    'segment-challenge.arff')  # use load_file to load from file instead
trainset.class_is_last()

testset = loader.load_url(weka_data_path + 'segment-test.arff')
testset.class_is_last()

### Random Forest

The following trains a random forest of 10 decision trees with maximum depth 5.

In [ ]:
from sklearn import ensemble

RF = ensemble.RandomForestClassifier(max_depth=5, 
                                     n_estimators=10, 
                                     random_state=0)
RF.fit(X_train, Y_train)
print(f'Accuracy: {RF.score(X_test, Y_test):.4g}')

**Exercise** Complete the pandas `DataFrame` `RF_df` in the following cell by filling in the accuracies (as floating point numbers) for different `n_estimators` and `max_depth`. Use `random_state = 0`.

In [ ]:
depth_list = [1, 2, 3, 5, 10, 20]
n_estimators_list = [1, 2, 3, 5, 10, 20, 30, 50, 100, 200, 300]
RF_df = pd.DataFrame(columns=[f'max_depth={max_depth}' for max_depth in depth_list], dtype=float)
RF_df.insert(0, 'n_estimators', n_estimators_list)
# YOUR CODE HERE
raise NotImplementedError()
display.display(RF_df.round(4))

plt.figure(figsize=(8, 5))
for col in RF_df.columns[1:]:
    plt.plot(RF_df['n_estimators'],
             RF_df[col],
             label=col,
             marker='o')
plt.legend()
plt.xlabel('n_estimators')
plt.ylabel('Accuracies')
plt.title(r'Random forest of different sizes and depths')
plt.show()

In [ ]:
# tests

To train a random forest of 10 decision trees with maximum depth 5 using `python-weka-wrapper`:

In [ ]:
RF = Classifier(classname="weka.classifiers.trees.RandomForest")
RF.options = ['-I', '10',
              '-depth', '5',
              '-S', '1']
RF.build_classifier(trainset)
evl = Evaluation(testset)
evl.test_model(RF, testset)
print(f'Accuracy {evl.percent_correct/100:.4g}')

**Exercise** Repeat the previous exercise but with Weka instead. Use a random seed of 1.

In [ ]:
depth_list = [1, 2, 3, 5, 10, 20]
n_estimators_list = [1, 2, 3, 5, 10, 20, 30, 50, 100, 200, 300]
RF_df = pd.DataFrame(columns=[f'max_depth={max_depth}' for max_depth in depth_list], dtype=float)
RF_df.insert(0, 'n_estimators', n_estimators_list)
# YOUR CODE HERE
raise NotImplementedError()
display.display(RF_df.round(4))

plt.figure(figsize=(8, 5))
for col in RF_df.columns[1:]:
    plt.plot(RF_df['n_estimators'],
             RF_df[col],
             label=col,
             marker='o')
plt.legend()
plt.xlabel('n_estimators')
plt.ylabel('Accuracies')
plt.title(r'Random forest of different sizes and depths')
plt.show()

In [ ]:
# tests

**Exercise** What are the best choices of `n_estimators` and `max_depth`?

YOUR ANSWER HERE

### AdaBoost

To train AdaBoost with 10 decision trees of maximum depth 5:

In [ ]:
ADB = ensemble.AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=5), n_estimators=10, random_state=0)
ADB.fit(X_train, Y_train)
print(f'Accuracy: {ADB.score(X_test, Y_test):.4g}')

**Exercise** Complete the pandas `DataFrame` `ADB_df` in the following cell by filling in the accuracies (as floating point numbers) for different `n_estimators` and `max_depth`. Use `random_state = 0`.

In [ ]:
depth_list = [1, 2, 3, 5, 10]
n_estimators_list = [1, 2, 3, 5, 10, 20, 30, 50, 100]
ADB_df = pd.DataFrame(columns=[f'max_depth={max_depth}' for max_depth in depth_list], dtype=float)
ADB_df.insert(0, 'n_estimators', n_estimators_list)
# YOUR CODE HERE
raise NotImplementedError()
display.display(ADB_df.round(4))

plt.figure(figsize=(8, 5))
for col in ADB_df.columns[1:]:
    plt.plot(ADB_df['n_estimators'],
             ADB_df[col],
             label=col,
             marker='o')
plt.legend()
plt.xlabel('n_estimators')
plt.ylabel('Accuracies')
plt.title(r'Adaboost of different sizes and depths')
plt.show()

In [ ]:
# tests

To train AdaBoost with 10 decision trees of maximum depth 5 using `python-weka-wrapper`:

In [ ]:
REPTree = Classifier(classname="weka.classifiers.trees.REPTree")
REPTree.options = ['-L', '5']
ADB = SingleClassifierEnhancer(classname="weka.classifiers.meta.AdaBoostM1")
ADB.options=['-I', '10',
             '-S', '1']
ADB.classifier = REPTree
ADB.build_classifier(trainset)
evl = Evaluation(testset)
evl.test_model(ADB, testset)
print(f'Accuracy {evl.percent_correct/100:.4g}')

**Exercise** Repeat the previous exercise but with Weka instead. Use a random seed of 1.

In [ ]:
depth_list = [1, 2, 3, 5, 10]
n_estimators_list = [1, 2, 3, 5, 10, 20, 30, 50, 100]
ADB_df = pd.DataFrame(columns=[f'max_depth={max_depth}' for max_depth in depth_list], dtype=float)
ADB_df.insert(0, 'n_estimators', n_estimators_list)
# YOUR CODE HERE
raise NotImplementedError()
display.display(ADB_df.round(4))

plt.figure(figsize=(8, 5))
for col in ADB_df.columns[1:]:
    plt.plot(ADB_df['n_estimators'],
             ADB_df[col],
             label=col,
             marker='o')
plt.legend()
plt.xlabel('n_estimators')
plt.ylabel('Accuracies')
plt.title(r'Adaboost of different sizes and depths')
plt.show()

In [ ]:
# tests

**Exercise** Which ensemble method is better, Adaboost or random forest? Why?

YOUR ANSWER HERE

## Optional Challenge

Train your own classifier to achieve the highest possible accuracies. You may:
- choose different classification algorithms or ensemble methods such as Bagging, Stacking, Voting, XGBoost, etc.
- tune the hyper-parameters manually or automatically using `GridSearchCV` in `scikit-learn` or `CVParameterSelection` in Weka.

Post your model and results on [Canvas](https://canvas.cityu.edu.hk/courses/39808/discussion_topics/306324) to compete with others.

The following is an example using XGBoost with its default parameters.
```Python
import xgboost
XGB = xgboost.XGBClassifier()
XGB.fit(X_train, Y_train)
print(f'Accuracy: {XGB.score(X_test, Y_test):.4g')
```